# 3. Architecture

In [1]:
# ⚔️ Runnable Code
import syft as sy
import hagrid 

sy.requires("==0.7")

✅ The installed version of syft==0.7.0 matches the requirement ==0.7


## Domain

Launching a domain: Better run the command below in a terminal and with `--dev` flag

In [ ]:
!hagrid launch test_domain domain to docker:8081 --tag=0.7.0 --tail

In [34]:
hagrid.check("localhost:8081", timeout=120)

Output()

✅ test_domain Domain Containers Created

✅ Backend

✅ Startup Complete

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━┓
┃ PyGrid    ┃ Info                                      ┃ 600 ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━┩
│ UI (βeta) │ http://localhost:8081/login               │ ❌  │
│ api       │ http://localhost:8081/api/v1/openapi.json │ ✅  │
└───────────┴───────────────────────────────────────────┴─────┘

In [29]:
domain_client = sy.login(
    port=8081,
    email="info@openmined.org",
    password="changethis"
)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into test_domain... done!


In [30]:
domain_client.name

'test_domain'

In [31]:
domain_client.routes[0].connection.base_url

<GridURL http://localhost:8081/api/v1>

In [32]:
import numpy as np
x = np.array([1, 2, 3])
x_ptr = x.send(domain_client)
x_ptr

<ndarrayPointer -> test_domain:1d74b417032f467094397495e426bea6, status=Processing>

In [33]:
t = sy.Tensor(x)
t_ptr = t.send(domain_client, tags=["Cool Data"])
t_ptr

Uploading `55b71902f0704972a41d6314fc69168d`: 100%|███████████████████| 1/1 [00:00<00:00, 19.96it/s]


<TensorPointer -> test_domain:55b71902f0704972a41d6314fc69168d, status=Processing>

## Network

In [ ]:
!hagrid launch test_network network to docker:8082 --tag=0.7.0 --tail

In [35]:
hagrid.check("localhost:8082", timeout=120)

Output()

✅ test_network Network Containers Created

✅ Backend

✅ Startup Complete

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━┓
┃ PyGrid    ┃ Info                                      ┃ 600 ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━┩
│ UI (βeta) │ http://localhost:8082/login               │ ✅  │
│ api       │ http://localhost:8082/api/v1/openapi.json │ ✅  │
└───────────┴───────────────────────────────────────────┴─────┘

Make a client for the network

In [36]:
network_client = sy.login(port=8082)

Connecting to localhost... done! 	 Logging into test_network... as GUEST...done!


Tell the domain to exchange its `verify_key` with the network

In [37]:
response = domain_client.networking.initiate_exchange_credentials(
    client=network_client
)

In [38]:
response

<InitiateExchangeCredentialsWithNodeMessage: 40fd2e3c1f2645c0a4af0122ba3ebc82>

Give the `network` knowledge of the `domain` by what `route` to connect 

In [39]:
response = domain_client.networking.add_route_for(
    client=network_client,
    source_node_url="http://localhost:8081",
    autodetect=False
)
response

<InitiateRouteUpdateToNodeMessage: d3e48981d16b4aa982328e6044d867df>

Ask the network to give a list of all its linked domains

In [40]:
network_client.domains

,host_or_ip,id,is_vpn,name,port,private,protocol
0,localhost,25028c4be7a745828f2ad8d1e6d6c64f,0,test_domain,8081,0,http


The purpose of registering a `domain` with a `network` is to provide data search and discovery

In [41]:
network_client.search(["Cool Data"])

,id,name,host_or_ip,is_vpn,private,protocol,port
0,25028c4be7a745828f2ad8d1e6d6c64f,test_domain,localhost,False,False,http,8081


We can also ask the `network` to proxy our commands via its own `private` connection to this `domain`

In [43]:
proxy_client = network_client.domains["25028c4be7a745828f2ad8d1e6d6c64f"]
proxy_client

(This is a logged out ProxyClient() object for a domain called 'test_domain'. Please call .login(email, password) to get a full client you can use for stuff.)

Check the `domain` store using the new proxy client

In [44]:
proxy_client.store

,ID,Tags,Description,object_type
0,<UID: d3db062762a040aa89be2ae6417f6f37>,[],,<class 'numpy.ndarray'>
1,<UID: 55b71902f0704972a41d6314fc69168d>,[Cool Data],,<class 'syft.core.tensor.tensor.Tensor'>
2,<UID: 1d74b417032f467094397495e426bea6>,[],,<class 'numpy.ndarray'>
3,<UID: 6c6bb78197c54af8b9db8e93bc14d789>,[Cool Data],,<class 'syft.core.tensor.tensor.Tensor'>


Check that the `Cool Data` is available

In [45]:
cool_data_uid = "55b71902f0704972a41d6314fc69168d"
cool_ptr = proxy_client.store[cool_data_uid]
cool_ptr

<TensorPointer -> test_domain:55b71902f0704972a41d6314fc69168d, status=Ready>

Getting the `Cool Data`

In [22]:
# ⚔️ Runnable Code
try:
    cool_ptr.get(delete_obj=False)
except Exception:
    print("You do not have permission to .get() \
          Object on the node. Please submit a request.")

[2023-02-02T22:47:49.731343+0700][CRITICAL][logger]][70289] Address unknown - cannot forward message. Throwing it away.


You do not have permission to .get()           Object on the node. Please submit a request.


This should fail because the `proxy_client` does not have permission. However, witht the authorized `domain_client`, we can get the data like in the code below

In [46]:
cool_ptr = domain_client.store[cool_data_uid]
cool_data = cool_ptr.get(delete_obj=False)
cool_data

Tensor(child=[1 2 3])

### Tailscale & Headscalei

`Tailscale`: An end-to-end encrypted VPN.  
`Headscale`: An open source implementation for key sharing by `tailscale`

In [47]:
domain_client.apply_to_network(network_client)

[1/4] Checking Syft Versions                            
[2/4] Joining Network                     
[3/4] Connecting to Secure VPN                              


[4/4] Registering on the Secure VPN                                   


## Trial of Insight

Start a new domain with name `khoa_r3q1`

In [ ]:
!hagrid launch khoa_r3q1 domain to docker:8081 --tag=0.7.0 --tail --dev

In [15]:
hagrid.check("localhost:8081")

Output()

✅ khoa_nguyen_r3q1 Domain Containers Created

✅ Backend

✅ Startup Complete

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━┓
┃ PyGrid    ┃ Info                                      ┃ 600 ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━┩
│ UI (βeta) │ http://localhost:8081/login               │ ❌  │
│ api       │ http://localhost:8081/api/v1/openapi.json │ ✅  │
└───────────┴───────────────────────────────────────────┴─────┘

In [3]:
domain_client = sy.login(
    port=8081,
    email="info@openmined.org",
    password="changethis"
)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into khoa_nguyen_r3q1... done!


In [16]:
domain_client

<DomainClient - khoa_nguyen_r3q1: <UID: 5dea5e0b2735436688ee06ea4fa7a6f1>>

Create some secret data in a `Syft Tensor` and sends it to the domain with `tags=["Kenobi"]`

In [27]:
import numpy as np
x = np.array([28, 10, 95])
t = sy.Tensor(x)
t_ptr = t.send(domain_client, tags=["Kenobi_1"])
t_ptr

Uploading `a94cccb04ea643b8992fa98f79a0b229`: 100%|██████████████████| 1/1 [00:00<00:00, 143.75it/s]


<TensorPointer -> khoa_nguyen_r3q1:a94cccb04ea643b8992fa98f79a0b229, status=Processing>

In [28]:
domain_client.store

,ID,Tags,Description,object_type
0,<UID: a94cccb04ea643b8992fa98f79a0b229>,[Kenobi_1],,<class 'syft.core.tensor.tensor.Tensor'>
1,<UID: 2a257cc0fdfe433a85a3d7e97099860b>,[Kenobi],,<class 'syft.core.tensor.tensor.Tensor'>


Locate the hidden rebel base

In [4]:
network_client = sy.login(url="20.253.234.47", port=80)

Connecting to 20.253.234.47... done! 	 Logging into hoth... as GUEST...done!

**Warning**: The syft version on your system and the node are different.
Version on your system: 0.7.0
Version on the node: 0.7.0-beta.62



Connect the `domain` to the rebel's VPN network

In [ ]:
domain_client.apply_to_network(network_client)

Domain: 0.7.0
Network: 0.7.0-beta.62
Environment: 0.7.0
[1/4] Checking Syft Versions                            
[2/4] Joining Network                     
[3/4] Connecting to Secure VPN                              


[4/4] Registering on the Secure VPN                                   


Check if the `network` has the `domain` registered

In [6]:
network_client.domains

,host_or_ip,id,is_vpn,name,port,private,protocol
0,100.64.0.2,855121fbf85a4855b09c91be2607a661,1,hpc_domain,80,0,http
1,100.64.0.3,f08babdeeaca4dfdbe6ba274f64005a7,1,hpc_domain2,80,0,http
2,100.64.0.4,c49ba0b5a39c48f5bdef902ca036b6ab,1,letv3_r3q1,80,0,http
3,100.64.0.5,eaf5f5f5e91d496e91f77d0f4433468d,1,khoa_r3q1,80,0,http
4,100.64.0.6,6f02c98a928247eb9915ac0bf3bb07d1,1,julian_r3q1,80,0,http
5,100.64.0.7,5dea5e0b2735436688ee06ea4fa7a6f1,1,khoa_nguyen_r3q1,80,0,http


Tinkering around

In [18]:
response = domain_client.networking.initiate_exchange_credentials(
    client=network_client
)
response

<InitiateExchangeCredentialsWithNodeMessage: 488d9639fb0c47afa7b847eddf641fab>

In [ ]:
response = domain_client.networking.add_route_for(
    client=network_client,
    source_node_url="http://localhost:8081",
    autodetect=False
)
response

In [20]:
domain_client.vpn_status()

{'status': 'ok',
 'connected': True,
 'host': {'ip': '100.64.0.7',
  'hostname': 'khoa-nguyen-r3q1',
  'network': 'omnet',
  'os': 'linux',
  'connection_info': '-',
  'connection_status': 'n/a',
  'connection_type': 'n/a'},
 'peers': [{'ip': '100.64.0.1',
   'hostname': 'hoth',
   'network': 'omnet',
   'os': 'linux',
   'connection_info': 'active; direct 20.253.234.47:41641; offline, tx 13604 rx 10308',
   'connection_status': 'active',
   'connection_type': 'direct'},
  {'ip': '100.64.0.2',
   'hostname': 'hpc-domain',
   'network': 'omnet',
   'os': 'linux',
   'connection_info': 'offline',
   'connection_status': 'n/a',
   'connection_type': 'n/a'},
  {'ip': '100.64.0.3',
   'hostname': 'hpc-domain2',
   'network': 'omnet',
   'os': 'linux',
   'connection_info': 'offline',
   'connection_status': 'n/a',
   'connection_type': 'n/a'},
  {'ip': '100.64.0.6',
   'hostname': 'julian-r3q1',
   'network': 'omnet',
   'os': 'linux',
   'connection_info': 'offline',
   'connection_status'

In [ ]:
network_client.search("[Kenobi]", timeout=1000000)

In [30]:
domain_client.store

,ID,Tags,Description,object_type
0,<UID: a94cccb04ea643b8992fa98f79a0b229>,[Kenobi_1],,<class 'syft.core.tensor.tensor.Tensor'>
1,<UID: 2a257cc0fdfe433a85a3d7e97099860b>,[Kenobi],,<class 'syft.core.tensor.tensor.Tensor'>
